<a href="https://colab.research.google.com/github/NicoZenith/Transformers/blob/main/GPT_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

from transformers import AutoTokenizer

import torch
import torch.nn as nn
from torch.nn import functional as F
import math

from transformer_layers import * 
from utils import *

/Users/deperrois/PycharmProjects/Transformers/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import dataset 
We will work with a text dataset containing all shakespeare pieces. 

In [2]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

response = requests.get(url)
text = response.text


In [3]:
print("length of dataset in characters", len(text))

length of dataset in characters 1115394


In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## GPT architecture 

We first construct the GPT architecture with the class `GPTLanguageModel`. It uses the layers defined in the `transformer_layers.py` file, and uses a forward mask as used in the Decoder of the original transformer, to prevent from looking ahead. 
It defines a forward method for next token prediction, and a generate method for generating tokens in a loop. 

In [5]:
class GPTLanguageModel(nn.Module):

    def __init__(self, d_model, num_heads, dropout, max_len, vocab_size, num_layers, device):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table

        self.max_len = max_len
        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        # self.position_embedding_table = nn.Embedding(max_len, n_embd)
        self.position_embedding_sinus = PositionalEncoding(d_model, p_drop=0.1, max_len=max_len, device=device)
        self.blocks = nn.Sequential(*[Block( d_model, num_heads, dropout) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(d_model) # final layer norm
        self.lm_head = nn.Linear(d_model, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, valid_lens=None, forward_mask=None, targets=None):
        """
        Takes a context sequence of tokens as input, and generates the next token. Can be used only for prediction (targets = None) 
        or for training.
        :idx: context sequence of tokens 
        :valid_lens: actual length of the sequence
        :forward_mask: for unidirectional attention 
        :targets: batch of targets for the next token
        """
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        # pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = self.position_embedding_sinus(tok_emb) # (B,T,C)
        for blk in self.blocks:
            x = blk(x, valid_lens, forward_mask) # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

        

    def generate(self, idx, max_new_tokens, valid_lens, forward_mask):
        """
        Generate text by taking a context of size max_len and predicting the next token 
        by sampling from the probability distribution obtained with the softmax on the whole vocabulary
        :idx: (B, T) array of indices in the current context
        :max_new_tokens: the number of new tokens we want to generate
        :return: generated sequence including the initial context
        """
        for _ in range(max_new_tokens):
            # crop idx to the last max_len tokens
            idx_cond = idx[:, -self.max_len:]
            # get the predictions
            logits, loss = self(idx_cond, valid_lens, forward_mask)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


    def generate_beam(self, idx, max_new_tokens, beam_size, valid_lens, forward_mask):
        """
        Generate text using beam search
        :beam_size: number of candidates at each beam search iteration
        """

        idx_cond = idx[:, -self.max_len:]
        logits, loss = self(idx_cond, valid_lens, forward_mask)
        # focus only on the last time step
        logits = logits[:,-1,:] # (B, C)
        softmax = nn.Softmax(dim=-1)
        probs = softmax(logits) # (B, C)

        candidates_idx_next = torch.topk(probs, beam_size, dim=1).indices # (B, k)
        # we create k different sequences, that concatenate each candidate
        store_sequences = []
        for i in range(beam_size):
            sequence = torch.cat([idx, candidates_idx_next[:, i].unsqueeze(1)], dim=1)
            sequence = torch.unsqueeze(sequence, 1)  # Add an extra dimension at dim=1
            store_sequences.append(sequence)
        all_idx = torch.cat(store_sequences, dim=1)

        sequence_probs = torch.topk(probs, beam_size, dim=1).values

        for _ in range(1, max_new_tokens):

            all_idx_cond = all_idx[:, :, -self.max_len:]

            store_topk_probs = []
            store_topk_indices = []

            for i in range(beam_size):
                logits, _ = self(all_idx_cond[:, i, :])
                logits = logits[:,-1,:] # (B, C)
                probs = softmax(logits)
                topk_probs = torch.topk(probs, beam_size, dim=1).values
                topk_indices = torch.topk(probs, beam_size, dim=1).indices
                store_topk_probs.append(topk_probs)
                store_topk_indices.append(topk_indices)

            # compute all probabilities of sequences
            all_topk_probs = torch.cat(store_topk_probs, dim=1) # contains the k**2 probabilities
            all_topk_indices = torch.cat(store_topk_indices, dim=1) # contains the k**2 indices

            sequence_probs = sequence_probs.repeat_interleave(beam_size, dim=1) # repeat the probabilities k times so we multiply with the new probas
            sequence_probs = sequence_probs * all_topk_probs # multiply to obtain probability of sequence of all selected leaves
            store = torch.topk(sequence_probs, beam_size, dim=1).indices
            idx_topk = all_topk_indices[torch.arange(probs.shape[0]).unsqueeze(1), store]
            sequence_probs = torch.topk(sequence_probs, beam_size, dim=1).values
            # we have the top k indices
            store_sequences = []
            for i in range(beam_size):
                index_for_previous_sequence = store[:,i] // probs.shape[1]
                previous_sequence_to_keep = all_idx[torch.arange(probs.shape[0]), index_for_previous_sequence]
                new_sequence = torch.cat([previous_sequence_to_keep, idx_topk[:,i].unsqueeze(1)], dim=1 )
                new_sequence = torch.unsqueeze(new_sequence, 1)
                store_sequences.append(new_sequence)
            all_idx = torch.cat(store_sequences, dim=1)

        # at the end of the loop, return the indices that have the best probability of sequence
        best_sequence = torch.argmax(sequence_probs, dim=1)
        idx = all_idx[torch.arange(probs.shape[0]), best_sequence]

        return idx

## Tokenization 

We will know create a mapping from tokens to a dictionary token ids necessary to pre-process the text sequences. 

In [6]:
use_tokenizer = True
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

if use_tokenizer:
    # create a mapping from text to word tokens
    model_name = "bert-base-uncased"  # Replace with the desired pre-trained model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Add a custom rule to handle line breaks
    text = text.replace('\n', '[')
    encode = lambda s: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s))
    decode = lambda l: tokenizer.convert_tokens_to_string([token.replace('[', '\n') if token == '[' \
                                                           else token for token in tokenizer.convert_ids_to_tokens(l)])
    vocab_size = tokenizer.vocab_size
else:
    # create a mapping from characters to integers
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(chars) }
    encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
    decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string   
    vocab_size = len(stoi) 



# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]



Token indices sequence length is longer than the specified maximum sequence length for this model (328719 > 512). Running this sequence through the model will result in indexing errors


## Hyperparameters

In [7]:


# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
max_len = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 250
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
d_model = 24
num_heads = 1
num_layers = 1
dropout = 0.2

# ------------
torch.manual_seed(1337)


## Training 

In [1]:

model = GPTLanguageModel(d_model, num_heads, dropout, max_len, vocab_size, num_layers, device)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model, eval_iters, train_data, val_data, batch_size, max_len, device)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(train_data, batch_size, max_len, device)

    # evaluate the loss
    logits, loss = model(xb, forward_mask=True, targets=yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()





NameError: name 'GPTLanguageModel' is not defined

## Generation 
After training the model, we generate some text starting from a context (here, with token ID 0). We can either generate with a greedy prediction (method generate) or with beam search (generate_beam)

In [9]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate_beam(context, max_new_tokens=300, beam_size=6, valid_lens=None, forward_mask=True)[0].tolist()))

KeyboardInterrupt: 